In [3]:
import MeCab
import unidic
print(unidic.DICDIR)
MeCab.Tagger("-d" + unidic.DICDIR)

/home/yamada/.local/lib/python3.6/site-packages/unidic/dicdir


<Swig Object of type 'MeCab::Tagger *' at 0x7f260e931e08>

- よくわからないもの

    
    

In [4]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

In [5]:
ngram_path = "../../corpus/google/3-gram/"

In [6]:
key1_set = set( os.listdir(ngram_path) ) 

In [7]:
os.listdir(ngram_path)[:10]

['ngram_ゴストン.json',
 'ngram_オーバーニーガール.json',
 'ngram_リアテーパー.json',
 'ngram_サブスポ.json',
 'ngram_リル・フリップ.json',
 'ngram_テリーヌサラダ.json',
 'ngram_リアルタイム・リミックス.json',
 'ngram_ナゼダロウ.json',
 'ngram_ヨネン.json',
 'ngram_倥.json']

In [8]:
path = "../eval_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

error = "Grammatical error"
errors = ['Grammatical error', "Uninterpretable"]
sys_utt = []
y = []
for conv in convs:
    for ut in conv:
        if ut.is_system() and ut.is_exist_error():
            if not ut.utt[-1] in ["？", "！", "。", "!"]:
                sys_utt.append( clean_text( ut.utt+"。" ))
                # sys_utt.append(ut.utt)
            else:   
                sys_utt.append(ut.utt)
            if ut.is_error_included(errors):
                y.append(1)
            else:
                y.append(0)

In [9]:
print("len of y:{0}, error '{1}' counts:{2}".format(len(y), errors, y.count(1)))

len of y:1386, error '['Grammatical error', 'Uninterpretable']' counts:7


In [10]:
def load_dict(path, key1):
    filename = key_formated(key1)
    with open(path+filename, "r") as f:
        key1_dict = json.load(f) 
    return key1_dict  

In [11]:
def key_formated(key1):
    return "ngram_"+key1+".json"

In [25]:
key2dict = dict()
registerform_dict = dict()
analyzer_unidic=  Analyzer(Tokenizer())

In [60]:
#  を考慮
# 何故か↑が出てくる
def judge_key3(key1_dict, ngram):
    key1 = ngram[0]
    key2 = ngram[1]
    key3 = ngram[2]

    if key2 not in key1_dict[key1]:
        # '"'をkeyにして聞いてみる
        if '"' in key1_dict[key1] and key2 in key1_dict[key1]['"']:
            return True
        if '、' in key1_dict[key1] and key2 in key1_dict[key1]['、']:
            return True
        return False
    
    if key3 not in key1_dict[key1][key2]:
        return False

    return True 

In [61]:

def search_registerforam(key, reading):
    filename = key_formated(key)
    if filename in key1_set:
        return key
    # 登録されていない
    else:
        for other in reading:
            filename = key_formated(key)
            if filename in key1_set:
                registerform_dict[key] = other
                return other
        return key


def sentence_judge(sentence, N=3):
    
    df = analyzer_unidic.analyze_with_dataframe(sentence)
    
    # 登録
    sentence_ = ""
    for txt, read in zip(df.surface, df.reading):
        sentence_ += search_registerforam(txt, read)
    sents = sentence2docs(sentence_, sents_span=True)
    # print(sents)

    # print(tokens)
    ngram_text = []
    ngram_pos = []

    for sent in sents:
        # 各文章 : sent
        df = analyzer_unidic.analyze_with_dataframe("".join([token.orth_ for token in sent]))
        surfaces = []
        poses = []
        readings = []
        for txt, p, read in zip(df.surface, df.part_of_speech, df.reading):
            surfaces.append(txt)
            readings.append(read)
            poses.append(p)

            for i in range(len(surfaces)-N+1):
                # print(L[i:i+N])
                ngram_text.append(surfaces[i:i+N])
                ngram_pos.append(poses[i:i+N])
                # ngram_reading.append(readings[i:i+N])
    
    # print(ngram_text)
    # return
    
    for ngram, pgram in zip(ngram_text, ngram_pos):
        # if "助動詞" in pgram[1] or "助詞" in pgram[1] or "助動詞" in pgram[0] or "助詞" in pgram[0]:
        if "助動詞" in pgram[0] or "助詞" in pgram[0]:
            # print("\tcontext : {0}| ->".format(ngram))

            key1 = ngram[0]
            if key_formated(key1) not in key1_set:
                continue

            if key1 in key2dict:
                key1_dict = key2dict[key1]
            else:
                key1_dict = load_dict(ngram_path, key1)
                key2dict[key1] = key1_dict

            # チェック
            if not judge_key3(key1_dict, ngram):
                print(ngram)
                # reading を調整
                
                # 見つかれば，その語彙を登録
                return False
    return True
            


In [62]:
# df = analyzer_unidic.analyze_with_dataframe("")
for name in list(key1_set):
    if "_渾名" in name:
        print(name)

ngram_渾名.json


In [63]:
key1_dict = load_dict(ngram_path, "を")

In [64]:
for k2 in key1_dict['を']:
    if "食べる" in key1_dict["を"][k2]:
        print(k2)

"
(
)
,
-
...
<UNK>
>
[
~
“
”
、
「
」
『
【
あえて
あさって
あっさり
あっという間に
あまり
ある程度
あれこれ
あんなに
あんまり
いかに
いきなり
いくつ
いざ
いただき
いち早く
いっきに
いっぱい
いっぺんに
いつ
いつか
いつも
いやいや
いよいよ
いれ
いろいろ
うっかり
うまく
うんと
おいしく
おおいに
おそるおそる
おとなしく
おもに
おもむろに
かけ
かける
かっこよく
かなり
かりかり
がっちり
がつがつ
がばがば
がぶがぶ
がりがり
がんがん
きちっと
きちんと
きっちり
くちゃくちゃ
けっこう
こう
こうして
こっそり
こりこり
これから
こんなに
ご飯
さくっと
さして
さっさと
さっそく
さっと
さっぱり
さらに
ざくざく
し
しいて
しこたま
しっかり
しばらく
しぶしぶ
しみじみ
しめて
しょっちゅう
じっくり
じゃんじゃん
すかさず
すぐ
すこし
すごく
すばやく
すべて
する
ずっと
ずるずる
ずーっと
せっせと
ぜひ
ぜんぶ
そこそこ
そっくり
そのまま
それぞれ
それだけ
それほど
そんなに
た
たいそう
たくさん
ただ
ただただ
たっぷり
たまに
たらふく
たんと
たんまり
だいぶ
だらだら
ちびちび
ちまちま
ちゃんと
ちょうど
ちょくちょく
ちょこちょこ
ちょこっと
ちょっと
ちょっぴり
ちょびちょび
ちょびっと
ついつい
ついに
つかみ
つくり
つくる
つけ
つける
つるつる
てんこもり
てんこ盛り
で
と
とうとう
ときどき
とことん
とても
とにかく
ともに
とり
とりあえず
とる
どう
どうして
どうしても
どうにか
どっさり
どんどん
なかなか
なぜ
なるべく
なんとか
なんとなく
ぬり
の
のせ
のんびり
はじめ
はじめて
ばりばり
ぱくぱく
ぱくりと
ぱりぱり
ひたすら
ひとくち
ひとしきり
ひとつ
ひとり
ひと通り
ふたつ
ふりかけ
ふんだんに
ぺろっと
ほくほく
ほとんど
ほどほど
ぼそぼそ
ぽんぽん
まず
まずく
また
まだ
まだまだ
まったく
まったり
まるごと
まるまる
まんべんなく
まんま
みんな
むき
むさぼり
むしゃむしゃ
むしり
むりやり
めちゃくちゃ
めったに
も
もう
もう一度
もう少し
もくもく
もくもくと
もぐもぐ
もしゃもしゃ
もそもそ
も

In [65]:
for k2 in key1_dict['を']['"']:
    print(k2)

"
"(
(
-
.
0
00
1
10
100
12
15
2
20
2003
24
3
30
4
5
50
587
6
7
8
9
</S>
<UNK>
>
A
AAA
AG
Ajax
B
British
C
CR
CRU
Cal
D
E
English
Express
F
False
From
G
GP
Gain
Gedo
Global
Google
H
HI
HTML
Hanefude
I
Impact
Incomplete
J
Japan
K
L
Lord
M
MS
MSGothic
Mr
My
N
No
O
OFF
OK
ON
OR
On
P
PC
PSP
PSPmk
PSmk
Q
R
S
SE
Shift
SignUp
Studio
Superstar
T
TEXT
The
U
UMD
UTF
V
VELO
We
Web
Windows
X
Y
YES
Yahoo
Yes
Z
a
anego
ape
b
bps
brother
c
ctrl
d
dat
dokdo
e
f
false
get
great
http
i
jp
m
mbk
mile
mush
my
n
new
no
nobody
none
nowa
o
off
on
p
postmaster
r
razo
razy
read
root
s
she
su
t
tabaco
test
the
true
txt
violet
x
y
yes
»
×
φ
、
あ
あい
あいつ
あえて
あきらめ
あげる
あっ
あと
あなた
あの
あり
ありがとう
ある
ある程度
あん
い
いい
いいえ
いかに
いけ
いたずら
いち
いつ
いつも
いま
いろいろ
う
うたう
うち
うっかり
うつけ
うなる
うまく
え
お
おいしい
おいしく
おじさん
おも
および
お前
お客様
お母さん
お気に入り
お父さん
お笑い
お金
か
かく
かけ
かける
かたち
かちり
かなり
かみ殺す
から
かわいい
かわいそう
が
き
きちんと
きれい
ぎゅっ
く
くさび
くじ
くすぐる
くらす
くん
ぐるぐる
けんか
こ
こう
ここ
こだわり
こちら
こっそり
こと
こども
この
これ
これから
こんな
ご
さ
さくら
さらに
さん
し
しっかり
しめつけ
しん
じ
じゃ
す
すき
すぐ
すっきり
すべて
する
せ
そう
そこ
そして

In [66]:
"食べる" in key1_dict["を"]

False

In [67]:
y_pred_2 = []
for i, utt in enumerate( tqdm( sys_utt ) )  :
    cleaned = clean_text(utt).replace("?", "。").replace("!", "")
    # print(cleaned)
    if sentence_judge(cleaned, N=3):
        y_pred_2.append(0)
    else:
        y_pred_2.append(1)
    # if i==35:
    #     break

  1%|          | 9/1386 [00:00<00:31, 43.23it/s]

['の', '逆回り', 'は']
['と', '、', '頼り甲斐']


  2%|▏         | 25/1386 [00:00<00:28, 47.55it/s]

['の', 'ソウル', 'らしい']
['を', '受け', 'ましょう']


  4%|▍         | 53/1386 [00:01<00:29, 44.87it/s]

...

['なんて', '個人', 'も']
['と', 'か', '暇']
['の', '踏み場', 'は']


 99%|█████████▉| 1377/1386 [00:43<00:00, 37.46it/s]

['の', '踏み場', 'は']
['は', '仕方ない', 'です']


100%|██████████| 1386/1386 [00:43<00:00, 31.99it/s]

['た', '飛行', '機']


In [68]:
y_pred_2.count(0)

893

In [69]:
score(y, y_pred_2)

confusion matrix = 
 [[887 492]
 [  6   1]]
accuracy =  0.6406926406926406
precision =  0.002028397565922921
recall =  0.14285714285714285
f1 score =  0.003999999999999999


- 使い方が難しい

        confusion matrix = 
        [[666 713]
        [  6   1]]
        accuracy =  0.48124098124098125
        precision =  0.0014005602240896359
        recall =  0.14285714285714285
        f1 score =  0.0027739251040221915

- ちょっと改善(無理やり)

        confusion matrix = 
        [[887 492]
        [  6   1]]
        accuracy =  0.6406926406926406
        precision =  0.002028397565922921
        recall =  0.14285714285714285
        f1 score =  0.003999999999999999

    - 意外と普通っぽい ngram がない
        - ["を", "覚える", *]

    - スコア化はせず， ngram が存在しなければアウトと仮定した
        - 先行研究のモデルで，そういったボーダーは存在しない

In [70]:
mecab_tokenize("お兄さんが死んだ")

['お', '兄さん', 'が', '死ん', 'だ']

In [71]:
df = analyzer_unidic.analyze_with_dataframe("頼りがいのあった可愛くない兄さんが死んだ")
df

,surface,part_of_speech,infl_type,infl_form,base_form,reading,phonetic
0,頼りがい,名詞-普通名詞-一般-,,,タヨリガイ,"[頼り甲斐, 頼りがい, タヨリガイ, 頼りがい, タヨリガイ]",*
1,の,助詞-格助詞--,,,ノ,"[の, の, ノ, の, ノ]",*
2,あっ,動詞-非自立可能--,五段-ラ行,連用形-促音便,アル,"[有る, あっ, アッ, ある, アル]",*
3,た,助動詞---,助動詞-タ,連体形-一般,タ,"[た, た, タ, た, タ]",*
4,可愛く,形容詞-一般--,形容詞,連用形-一般,カワイイ,"[可愛い, 可愛く, カワイク, 可愛い, カワイー]",*
5,ない,形容詞-非自立可能--,形容詞,連体形-一般,ナイ,"[無い, ない, ナイ, ない, ナイ]",*
6,兄,名詞-普通名詞-一般-,,,アニ,"[兄, 兄, アニ, 兄, アニ]",*
7,さん,接尾辞-名詞的-一般-,,,サン,"[さん, さん, サン, さん, サン]",*
8,が,助詞-格助詞--,,,ガ,"[が, が, ガ, が, ガ]",*
9,死ん,動詞-一般--,五段-ナ行,連用形-撥音便,シヌ,"[死ぬ, 死ん, シン, 死ぬ, シヌ]",*
